In [92]:
import numpy as np
import numpy.typing as npt
import pytest
import stim
from graphix import Circuit, Pattern, command
from graphix.fundamentals import Plane
#from graphix.noise_models.depolarising_noise_model import DepolarisingNoiseModel
#from graphix.noise_models.noise_model import NoiseModel
from graphix.random_objects import rand_circuit
from graphix.sim.base_backend import (
    FixedBranchSelector,
    RandomBranchSelector,
)
from graphix.simulator import DefaultMeasureMethod
from graphix.states import BasicStates
from numpy.random import PCG64, Generator
from veriphix.client import Client, Secrets
from veriphix.trappifiedCanvas import TrappifiedCanvas

from gospel.brickwork_state_transpiler import (
    ConstructionOrder,
    generate_random_pauli_pattern,
    get_bipartite_coloring,
)
from gospel.scripts import compare_backend_results
from gospel.stim_pauli_preprocessing import (
    StimBackend,
    cut_pattern,
    preprocess_pauli,
    simulate_pauli,
)

In [93]:
"""Uncorrelated depolarising noise model."""

from __future__ import annotations

from dataclasses import dataclass
from typing import TYPE_CHECKING

import typing_extensions
from graphix.channels import (
    KrausChannel,
    depolarising_channel,
    two_qubit_depolarising_tensor_channel,
)
from graphix.command import BaseM, CommandKind
from graphix.noise_models.noise_model import (
    A,
    CommandOrNoise,
    Noise,
    NoiseCommands,
    NoiseModel,
)
from graphix.rng import ensure_rng

if TYPE_CHECKING:
    from numpy.random import Generator
from graphix.noise_models.depolarising_noise_model import (
    DepolarisingNoise,
    TwoQubitDepolarisingNoise,
)

class UncorrelatedDepolarisingNoiseModel(NoiseModel):
    """Depolarising noise model.

    Only return the identity channel.

    :param NoiseModel: Parent abstract class class:`graphix.noise_model.NoiseModel`
    :type NoiseModel: class
    """

    def __init__(
        self,
        prepare_error_prob: float = 0.0,
        x_error_prob: float = 0.0,
        z_error_prob: float = 0.0,
        entanglement_error_prob: float = 0.0,
        measure_channel_prob: float = 0.0,
        measure_error_prob: float = 0.0,
        rng: Generator = None,
    ) -> None:
        self.prepare_error_prob = prepare_error_prob
        self.x_error_prob = x_error_prob
        self.z_error_prob = z_error_prob
        self.entanglement_error_prob = entanglement_error_prob
        self.measure_error_prob = measure_error_prob
        self.measure_channel_prob = measure_channel_prob
        self.rng = ensure_rng(rng)

    def input_nodes(self, nodes: list[int]) -> NoiseCommands:
        """Return the noise to apply to input nodes."""
        return [
            A(noise=DepolarisingNoise(self.prepare_error_prob), nodes=[node])
            for node in nodes
        ]

    def command(self, cmd: CommandOrNoise) -> NoiseCommands:
        """Return the noise to apply to the command `cmd`."""
        if cmd.kind == CommandKind.N:
            return [
                cmd,
                A(noise=DepolarisingNoise(self.prepare_error_prob), nodes=[cmd.node]),
            ]
        if cmd.kind == CommandKind.E:
            u, v = cmd.nodes
            return [
                cmd,
                A(
                    noise=DepolarisingNoise(
                        self.entanglement_error_prob
                    ),
                    nodes=[u],
                ),
                A(
                    noise=DepolarisingNoise(
                        self.entanglement_error_prob
                    ),
                    nodes=[v],
                ),
            ]
        if cmd.kind == CommandKind.M:
            return [
                A(noise=DepolarisingNoise(self.measure_channel_prob), nodes=[cmd.node]),
                cmd,
            ]
        if cmd.kind == CommandKind.X:
            return [
                cmd,
                A(noise=DepolarisingNoise(self.x_error_prob), nodes=[cmd.node]),
            ]
        if cmd.kind == CommandKind.Z:
            return [
                cmd,
                A(noise=DepolarisingNoise(self.z_error_prob), nodes=[cmd.node]),
            ]
        # Use of `==` here for mypy
        if (
            cmd.kind == CommandKind.C  # noqa: PLR1714
            or cmd.kind == CommandKind.T
            or cmd.kind == CommandKind.A
            or cmd.kind == CommandKind.S
        ):
            return [cmd]
        typing_extensions.assert_never(cmd.kind)

    def confuse_result(self, cmd: BaseM, result: bool) -> bool:
        """Assign wrong measurement result cmd = "M"."""
        if self.rng.uniform() < self.measure_error_prob:
            return not result
        return result

In [94]:
import numpy as np
from numpy.random import PCG64, Generator
from typing import List, Dict

# Assuming these classes and functions are imported properly
# from your library
# from your_library import Secrets, Client, get_bipartite_coloring, TrappifiedCanvas, StimBackend, DepolarisingNoiseModel, generate_random_pauli_pattern, command

# Initialization
fx_bg = PCG64(42)
jumps = 5
n_iterations = 5000  # Number of test iterations

# Define separate outcome tables for Canonical and Deviant
test_outcome_table_canonical: List[Dict] = []
test_outcome_table_deviant: List[Dict] = []

# Loop over Construction Orders
for order in (ConstructionOrder.Canonical, ConstructionOrder.Deviant):
    rng = Generator(fx_bg.jumped(jumps))  # Use the jumped rng
    pattern = generate_random_pauli_pattern(
        nqubits=8, nlayers=10, order=order, rng=rng
    )

    # Add measurement commands to the output nodes
    for onode in pattern.output_nodes:
        pattern.add(command.M(node=onode))

    # Initialize secrets and client
    secrets = Secrets(r=False, a=False, theta=False)
    client = Client(pattern=pattern, secrets=secrets)
    
    # Get bipartite coloring and create test runs
    colours = get_bipartite_coloring(pattern)
    test_runs = client.create_test_runs(manual_colouring=colours)

    # Define backend and noise model
    noise_model = UncorrelatedDepolarisingNoiseModel(entanglement_error_prob = 0.001)

    n_failures = 0

    # Choose the correct outcome table based on order
    if order == ConstructionOrder.Canonical:
        test_outcome_table = test_outcome_table_canonical
    else:
        test_outcome_table = test_outcome_table_deviant

    print(f"Running {n_iterations} iterations for order: {order}", flush=True)

    # Run the test iterations
    for i in range(n_iterations):
        backend = StimBackend()
        # Select a random test run
        run = TrappifiedCanvas(test_runs[rng.integers(len(test_runs))], rng=rng)

        # Delegate the test run to the client
        trap_outcomes = client.delegate_test_run(backend=backend, run=run, noise_model=noise_model)
        
        
        # Create a result dictionary (trap -> outcome)
        result = {
            tuple(trap): outcome for trap, outcome in zip(run.traps_list, trap_outcomes)
        }

        # Append the result to the appropriate test outcome table
        test_outcome_table.append(result)

        # Print pass/fail based on the sum of the trap outcomes
        if sum(trap_outcomes) != 0:
            n_failures += 1
            print(f"Iteration {i+1}: ❌ Failed trap round", flush=True)
        else:
            print(f"Iteration {i+1}: ✅ Trap round passed", flush=True)

    # Final report after completing the test rounds
    print(f"Final result for {order}: {n_failures}/{n_iterations} failed rounds", flush=True)
    print("-" * 50, flush=True)
print(f"Number of nodes in the pattern : {pattern.n_node}")
    # Uncomment this line if you want to assert no failures occurred
    # assert n_failures == 0, f"Test failed: {n_failures} trap rounds detected noise."

# Now you have two separate outcome tables:
# test_outcome_table_canonical for Canonical order
# test_outcome_table_deviant for Deviant order


Running 5000 iterations for order: ConstructionOrder.Canonical
Iteration 1: ✅ Trap round passed
Iteration 2: ✅ Trap round passed
Iteration 3: ✅ Trap round passed
Iteration 4: ✅ Trap round passed
Iteration 5: ✅ Trap round passed
Iteration 6: ❌ Failed trap round
Iteration 7: ❌ Failed trap round
Iteration 8: ✅ Trap round passed
Iteration 9: ❌ Failed trap round
Iteration 10: ❌ Failed trap round
Iteration 11: ✅ Trap round passed
Iteration 12: ✅ Trap round passed
Iteration 13: ❌ Failed trap round
Iteration 14: ✅ Trap round passed
Iteration 15: ❌ Failed trap round
Iteration 16: ✅ Trap round passed
Iteration 17: ✅ Trap round passed
Iteration 18: ✅ Trap round passed
Iteration 19: ✅ Trap round passed
Iteration 20: ✅ Trap round passed
Iteration 21: ✅ Trap round passed
Iteration 22: ❌ Failed trap round
Iteration 23: ✅ Trap round passed
Iteration 24: ❌ Failed trap round
Iteration 25: ✅ Trap round passed
Iteration 26: ✅ Trap round passed
Iteration 27: ❌ Failed trap round
Iteration 28: ✅ Trap round p

In [95]:
print(len(test_outcome_table_canonical))
occurences = {}
occurences_one = {}

for results in test_outcome_table_canonical:
    for q, r in results.items():
        if q not in occurences:
            occurences[q] = 1
            occurences_one[q] = r
        else:
            occurences[q] += 1
            if r == 1:
                occurences_one[q] += 1

failure_proba_can_final = {q: occurences_one[q] / occurences[q] for q in occurences}
#print(failure_proba_can)
failure_proba_can_array = [v for k, v in sorted(failure_proba_can_final.items(), key=lambda x: x[0][0])]
#print(failure_proba_can)

5000


In [96]:
print(len(test_outcome_table_deviant))
occurences = {}
occurences_one = {}

for results in test_outcome_table_deviant:
    for q, r in results.items():
        if q not in occurences:
            occurences[q] = 1
            occurences_one[q] = r
        else:
            occurences[q] += 1
            if r == 1:
                occurences_one[q] += 1

failure_proba_dev_all = {q: occurences_one[q] / occurences[q] for q in occurences}
#print(failure_proba_dev_all)
#print(failure_proba_dev_all.keys())  # Inspect key format

5000


In [97]:
# Generate the required indices as TUPLES
required_indices = []
start = 8
max_index = 327

while start <= max_index:
    for offset in [0, 2, 4, 6]:
        current_index = start + offset
        if current_index > max_index:
            break
        required_indices.append((current_index,))  # Note the comma to create tuple
    start += 16

# Extract with tuple keys and existence check
failure_proba_dev_final = {
    idx: failure_proba_dev_all[idx] 
    for idx in required_indices 
    if idx in failure_proba_dev_all
}

#print(failure_proba_dev_final)
failure_proba_dev_array = [v for k, v in sorted(failure_proba_dev_final.items(), key=lambda x: x[0][0])]
print(failure_proba_dev_array)
#print("Available indices:", [idx for idx in required_indices if idx in failure_proba_dev_all])

[0.002369668246445498, 0.002369668246445498, 0.001579778830963665, 0.002764612954186414, 0.004739336492890996, 0.00315955766192733, 0.005134281200631911, 0.004344391785150079, 0.0039494470774091624, 0.0035545023696682463, 0.005924170616113744, 0.005134281200631911, 0.001579778830963665, 0.002369668246445498, 0.004739336492890996, 0.005134281200631911, 0.002764612954186414, 0.0019747235387045812, 0.0039494470774091624, 0.00315955766192733, 0.002764612954186414, 0.0019747235387045812, 0.0035545023696682463, 0.002764612954186414, 0.0039494470774091624, 0.0035545023696682463, 0.0039494470774091624, 0.004739336492890996, 0.001184834123222749, 0.0019747235387045812, 0.00315955766192733, 0.002369668246445498, 0.002369668246445498, 0.0019747235387045812, 0.0035545023696682463, 0.004344391785150079, 0.00315955766192733, 0.002764612954186414, 0.0039494470774091624, 0.004344391785150079, 0.0019747235387045812, 0.002764612954186414, 0.00315955766192733, 0.004344391785150079, 0.001579778830963665, 

In [99]:
failure_proba_can_inverted = [1 - x for x in failure_proba_can_array]
failure_proba_dev_inverted = [1 - x for x in failure_proba_dev_array]
# Both lists are now available:
#print("can:", failure_proba_can_inverted)  
#print("dev:", failure_proba_dev_inverted)  

failure_proba_can = [abs(orig - inv) for orig, inv in zip(failure_proba_can_array, failure_proba_can_inverted)]
failure_proba_dev = [abs(origi - inve) for origi, inve in zip(failure_proba_dev_array, failure_proba_dev_inverted)]

#print("can:", failure_proba_can)  
#print("dev:", failure_proba_dev) 

In [100]:
import numpy as np
import sympy as sp


def generate_qubit_edge_matrix_with_unknowns(n, m):
    #assert n % 2 == 0, "The number of rows (n) must be even."

    qubits = {}  # Mapping from (i, j) to qubit index
    edges = {}   # Mapping from edge (start, end) to edge index
    edge_index = 0
    qubit_index = 0

    # Assign an index to each qubit (i, j)
    for j in range(m):
        for i in range(n):
            qubits[(i, j)] = qubit_index
            qubit_index += 1

    # Collect all edges and assign them an index
    for i in range(n):
        for j in range(m - 1):
            edge = ((i, j), (i, j + 1))  # Horizontal edge
            edges[edge] = edge_index
            edge_index += 1

    for i in range(n - 1):
        for j in range(m):
            if ((j + 1) % 8 == 3 and (i + 1) % 2 != 0):  # Column j ≡ 3 (mod 8) and odd row i
                if j + 3 < m:  # Ensure we don't go out of bounds
                    edge = ((i, j), (i + 1, j))
                    edges[edge] = edge_index
                    edge_index += 1
                    edge = ((i, j + 2), (i + 1, j + 2))
                    edges[edge] = edge_index
                    edge_index += 1
            if ((j + 1) % 8 == 7 and (i + 1) % 2 == 0):  # Column j ≡ 7 (mod 8) and even row i
                if j + 3 < m:  # Ensure we don't go out of bounds
                    edge = ((i, j), (i + 1, j))
                    edges[edge] = edge_index
                    edge_index += 1
                    edge = ((i, j + 2), (i + 1, j + 2))
                    edges[edge] = edge_index
                    edge_index += 1

    # Create the symbolic matrix (qubits × edges)
    matrix = np.zeros((len(qubits), len(edges)), dtype=object)

    # Create symbolic variables for edges
    edge_symbols = [sp.symbols(f'x{i}') for i in range(len(edges))]

    # Apply special conditions for qubits
    conditions = [
        (lambda i, j: ((i % 2 == 0 and (j % 8 == 0 or j % 8 == 6)) or (i % 2 == 1 and (j % 8 == 2 or j % 8 == 4)), [
            ((i, j - 2), (i, j - 1)),
            ((i, j - 1), (i, j)),
            ((i - 1, j - 1), (i - 1, j)),
            ((i - 1, j), (i, j)),
            ((i, j), (i, j + 1))
        ])),
        (lambda i, j: ((i % 2 == 1 and (j % 8 == 0 or j % 8 == 6)) or (i % 2 == 0 and (j % 8 == 2 or j % 8 == 4)), [
            ((i, j - 2), (i, j - 1)),
            ((i, j - 1), (i, j)),
            ((i + 1, j - 1), (i + 1, j)),
            ((i, j), (i + 1, j)),
             ((i, j), (i, j + 1))
        ])),
        (lambda i, j: ((i % 2 == 0 and (j % 8 == 1 or j % 8 == 7)) or (i % 2 == 1 and (j % 8 == 3 or j % 8 == 5)), [
            ((i, j - 2), (i, j - 1)),
            ((i - 1, j - 1), (i, j - 1)),
            ((i, j - 1), (i, j)),
            ((i, j), (i, j + 1))
        ])),
        (lambda i, j: ((i % 2 == 1 and (j % 8 == 1 or j % 8 == 7)) or (i % 2 == 0 and (j % 8 == 3 or j % 8 == 5)), [
            ((i, j - 2), (i, j - 1)),
            ((i, j - 1), (i + 1, j)),
            ((i, j - 1), (i, j)),
            ((i, j), (i, j + 1))
        ]))
    ]


    #print(edges)
    for f in conditions:
        for i in range(n):
            for j in range(m):
                condition, special_edges = f(i, j)
                if condition:
                    for (rel_i1, rel_j1), (rel_i2, rel_j2) in special_edges:
                        # Compute actual coordinates of edge
                        edge = ((rel_i1, rel_j1), (rel_i2, rel_j2))

                        # Ensure the edge exists before modifying the matrix
                        if edge in edges:
                            e_idx = edges[edge]
                            q_idx = qubits[(i, j)]
                            # Use symbolic variables for edges
                            #matrix[q_idx, e_idx] = edge_symbols[e_idx]
                            matrix[q_idx, e_idx] = 1
                        #else:
                            #print(edge)

    # Print the edge mapping (index to actual edge)
    print("\nEdge to Column Mapping:")
    #for edge, idx in edges.items():
        #print(f"Column {idx} corresponds to edge: {edge}")

    print("\nQubit to Row Mapping:")
    for qubit, idx in qubits.items():
        print(f"Row {idx} corresponds to qubit: {qubit}")

    # Return matrix with symbolic edge variables
    return matrix, qubits, edges, edge_symbols

# Define grid size
n, m = 8, 41  # n must be even

# Generate the matrix with unknown edge parameters
qubit_edge_matrix, qubit_map, edge_map, edge_symbols = generate_qubit_edge_matrix_with_unknowns(n, m)

#print("Number of node:", qubit_map)
# Display symbolic edge variables
#print("\nEdge Variables:")
#for i, symbol in enumerate(edge_symbols):
 #   print(f"x{i}: {symbol}")

# Display result
#print("Qubit-Edge Matrix (with symbolic edge parameters):")
#print(qubit_edge_matrix)



Edge to Column Mapping:

Qubit to Row Mapping:
Row 0 corresponds to qubit: (0, 0)
Row 1 corresponds to qubit: (1, 0)
Row 2 corresponds to qubit: (2, 0)
Row 3 corresponds to qubit: (3, 0)
Row 4 corresponds to qubit: (4, 0)
Row 5 corresponds to qubit: (5, 0)
Row 6 corresponds to qubit: (6, 0)
Row 7 corresponds to qubit: (7, 0)
Row 8 corresponds to qubit: (0, 1)
Row 9 corresponds to qubit: (1, 1)
Row 10 corresponds to qubit: (2, 1)
Row 11 corresponds to qubit: (3, 1)
Row 12 corresponds to qubit: (4, 1)
Row 13 corresponds to qubit: (5, 1)
Row 14 corresponds to qubit: (6, 1)
Row 15 corresponds to qubit: (7, 1)
Row 16 corresponds to qubit: (0, 2)
Row 17 corresponds to qubit: (1, 2)
Row 18 corresponds to qubit: (2, 2)
Row 19 corresponds to qubit: (3, 2)
Row 20 corresponds to qubit: (4, 2)
Row 21 corresponds to qubit: (5, 2)
Row 22 corresponds to qubit: (6, 2)
Row 23 corresponds to qubit: (7, 2)
Row 24 corresponds to qubit: (0, 3)
Row 25 corresponds to qubit: (1, 3)
Row 26 corresponds to qubi

In [101]:
import numpy as np
import sympy as sp


def generate_qubit_edge_matrix_with_unknowns_dev(n, m):
    #assert n % 2 == 0, "The number of rows (n) must be even."

    qubits_dev = {}  # Mapping from (i, j) to qubit index
    edges_dev = {}   # Mapping from edge (start, end) to edge index
    edge_index_dev = 0
    qubit_index_dev = 0

    # Assign an index to each qubit (i, j)
    for j in range(m):
        for i in range(n):
            if(i % 2 == 0 and (j % 8 == 1 or j % 8 == 3 or j % 8 == 5 or j % 8 == 7)):
                qubits_dev[(i, j)] = qubit_index_dev
                qubit_index_dev += 1

    # Collect all edges and assign them an index
    for i in range(n):
        for j in range(m - 1):
            edge_dev = ((i, j), (i, j + 1))  # Horizontal edge
            edges_dev[edge_dev] = edge_index_dev
            edge_index_dev += 1

    for i in range(n - 1):
        for j in range(m):
            if ((j + 1) % 8 == 3 and (i + 1) % 2 != 0):  # Column j ≡ 3 (mod 8) and odd row i
                if j + 3 < m:  # Ensure we don't go out of bounds
                    edge_dev = ((i, j), (i + 1, j))
                    edges_dev[edge_dev] = edge_index_dev
                    edge_index_dev += 1
                    edge_dev = ((i, j + 2), (i + 1, j + 2))
                    edges_dev[edge_dev] = edge_index_dev
                    edge_index_dev += 1
            if ((j + 1) % 8 == 7 and (i + 1) % 2 == 0):  # Column j ≡ 7 (mod 8) and even row i
                if j + 3 < m:  # Ensure we don't go out of bounds
                    edge_dev = ((i, j), (i + 1, j))
                    edges_dev[edge_dev] = edge_index_dev
                    edge_index_dev += 1
                    edge_dev = ((i, j + 2), (i + 1, j + 2))
                    edges_dev[edge_dev] = edge_index_dev
                    edge_index_dev += 1

    # Create the symbolic matrix (qubits × edges)
    matrix_dev = np.zeros((len(qubits_dev), len(edges_dev)), dtype=object)

    # Create symbolic variables for edges
    edge_symbols_dev = [sp.symbols(f'x{i}') for i in range(len(edges_dev))]

    # Apply special conditions for qubits
    conditions_dev = [
        (lambda i, j: ((i % 2 == 0 and j % 8 == 1), [
            ((i, j - 2), (i, j - 1)),
            ((i - 1, j - 1), (i, j - 1)),
            ((i, j - 1), (i, j)),
            ((i, j), (i, j + 1)),
            ((i, j + 1), (i + 1, j + 1))
        ])),
        (lambda i, j: ((i % 2 == 0 and j % 8 == 3), [
            ((i, j - 2), (i, j - 1)),
            ((i, j - 1), (i + 1, j - 1)),
            ((i, j - 1), (i, j)),
            ((i, j), (i, j + 1)),
             ((i, j + 1), (i + 1, j + 1))
        ])),
        (lambda i, j: ((i % 2 == 0 and j % 8 == 5), [
            ((i, j - 2), (i, j - 1)),
            ((i, j - 1), (i + 1, j - 1)),
            ((i, j - 1), (i, j)),
            ((i, j), (i, j + 1)),
            ((i - 1, j + 1), (i, j + 1))
        ])),
        (lambda i, j: ((i % 2 == 0 and j % 8 == 7), [
            ((i, j - 2), (i, j - 1)),
            ((i - 1, j - 1), (i, j - 1)),
            ((i, j - 1), (i, j)),
            ((i, j), (i, j + 1)),
            ((i - 1, j + 1), (i, j + 1))
        ]))
    ]


    #print(edges_dev)
    for f in conditions_dev:
        for i in range(n):
            for j in range(m):
                if(i % 2 == 0 and (j % 8 == 1 or j % 8 == 3 or j % 8 == 5 or j % 8 == 7)):
                    condition_dev, special_edges_dev = f(i, j)
                    if condition_dev:
                        for (rel_i1, rel_j1), (rel_i2, rel_j2) in special_edges_dev:
                            # Compute actual coordinates of edge
                            edge_dev = ((rel_i1, rel_j1), (rel_i2, rel_j2))

                            # Ensure the edge exists before modifying the matrix
                            if edge_dev in edges_dev:
                                e_idx_dev = edges_dev[edge_dev]
                                q_idx_dev = qubits_dev[(i, j)]
                                # Use symbolic variables for edges
                                #matrix_dev[q_idx_dev, e_idx_dev] = edge_symbols_dev[e_idx_dev]
                                matrix_dev[q_idx_dev, e_idx_dev] = 1
                            #else:
                                #print(edge_dev)

    # Print the edge mapping (index to actual edge)
    #print("\nEdge to Column Mapping:")
    #for edge_dev, idx in edges_dev.items():
        #print(f"Column {idx} corresponds to edge: {edge_dev}")

    print("\nQubit to Row Mapping:")
    for qubit_dev, idx in qubits_dev.items():
        print(f"Row {idx} corresponds to qubit: {qubit_dev}")

    # Return matrix with symbolic edge variables
    return matrix_dev, qubits_dev, edges_dev, edge_symbols_dev

# Define grid size
n, m = 8, 41  # n must be even

# Generate the matrix with unknown edge parameters
qubit_edge_matrix_dev, qubit_map_dev, edge_map_dev, edge_symbols_dev = generate_qubit_edge_matrix_with_unknowns_dev(n, m)


# Display symbolic edge variables
#print("\nEdge Variables:")
#for i, symbol in enumerate(edge_symbols_dev):
 #   print(f"x{i}: {symbol}")

# Display result
print("Qubit-Edge Matrix (with symbolic edge parameters):")
print(qubit_edge_matrix_dev)



Qubit to Row Mapping:
Row 0 corresponds to qubit: (0, 1)
Row 1 corresponds to qubit: (2, 1)
Row 2 corresponds to qubit: (4, 1)
Row 3 corresponds to qubit: (6, 1)
Row 4 corresponds to qubit: (0, 3)
Row 5 corresponds to qubit: (2, 3)
Row 6 corresponds to qubit: (4, 3)
Row 7 corresponds to qubit: (6, 3)
Row 8 corresponds to qubit: (0, 5)
Row 9 corresponds to qubit: (2, 5)
Row 10 corresponds to qubit: (4, 5)
Row 11 corresponds to qubit: (6, 5)
Row 12 corresponds to qubit: (0, 7)
Row 13 corresponds to qubit: (2, 7)
Row 14 corresponds to qubit: (4, 7)
Row 15 corresponds to qubit: (6, 7)
Row 16 corresponds to qubit: (0, 9)
Row 17 corresponds to qubit: (2, 9)
Row 18 corresponds to qubit: (4, 9)
Row 19 corresponds to qubit: (6, 9)
Row 20 corresponds to qubit: (0, 11)
Row 21 corresponds to qubit: (2, 11)
Row 22 corresponds to qubit: (4, 11)
Row 23 corresponds to qubit: (6, 11)
Row 24 corresponds to qubit: (0, 13)
Row 25 corresponds to qubit: (2, 13)
Row 26 corresponds to qubit: (4, 13)
Row 27 c

In [103]:
failure_proba_can = np.array(failure_proba_can, dtype=np.float64)
failure_proba_dev = np.array(failure_proba_dev, dtype=np.float64) 

qubit_edge_matrix = np.array(qubit_edge_matrix, dtype=np.float64)
qubit_edge_matrix_dev = np.array(qubit_edge_matrix_dev, dtype=np.float64)


# Stack the matrices together to form a single system
lhs = np.vstack((qubit_edge_matrix, qubit_edge_matrix_dev))  # Combine coefficient matrices
#print(lhs)
rhs = np.concatenate((failure_proba_can, failure_proba_dev))  # Combine constant vectors

#print(rhs)

log_rhs = np.log(rhs)  # log constant vectors

log_params, residuals, rank, singular_values = np.linalg.lstsq(lhs, log_rhs, rcond=None)

X = np.exp(log_params)  # Convert log values back to original variables

# Print the solution
print(f"Solution vectors:", X)

Solution vectors: [0.99746356 1.00159358 1.00179835 0.9964649  1.00291294 0.99836603
 0.99623274 0.99955326 1.0008466  0.99596997 0.99291675 1.00638426
 0.9997967  0.98845959 1.00532085 1.00280844 0.98536229 1.00795221
 0.98465932 1.00133454 1.00844856 0.98625731 0.99810664 1.01179428
 0.98871261 0.99543533 1.02046627 0.97909223 1.01036612 1.0027881
 0.98218944 1.01376077 0.99767988 0.98306628 1.01889006 0.99370582
 1.00503365 0.99647639 0.99689159 1.00149518 0.99897919 0.99709369
 1.00449169 0.99842653 0.99876717 1.00133899 1.00350583 0.98918195
 0.99655493 1.0079862  0.99411677 0.9948957  1.0013343  0.99765785
 1.01087646 0.98773429 0.99407072 1.01422806 0.97914044 1.00036961
 1.0119751  0.98103592 0.99703685 1.01601219 0.98684402 0.99020896
 1.02116961 0.98427997 1.00326694 1.00607689 0.98535206 1.00221267
 0.99415308 0.99633763 1.01846264 0.97776369 1.00014847 1.01576554
 0.96104223 1.01883757 0.99825558 0.99963668 0.99465234 1.00013519
 0.99863141 0.99569905 0.99933699 1.00282223 